In [114]:
###############################################################################################################
## Intermediate file for identifiers and variant information -- Working
# Module import

import pandas as pd
import os


# Preparation

relevant = ['Tumor_Sample_UUID','Matched_Norm_Sample_UUID','case_id', 'NCBI_Build','Chromosome',
          'Start_Position','End_Position','Strand', 'Variant_Classification', 'Variant_Type',
          'Reference_Allele', 'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'HGVSc', 'HGVSp',
          'HGVSp_Short', 'all_effects'] # combined identifiers and variant information

imf = pd.DataFrame() # Create empty dataframe for intermediate file

for info in relevant: # Assign columns from relevant list
    imf[info] = []

for file in os.listdir("../data/"): # Iterate through directory
    if '.maf' in str(file): # discard MANIFEST.txt and other non-maf files
        df = pd.read_csv("../data/"+ str(file), sep='\t', skiprows=7, header=0) # read in the maf file as df
        df = df[relevant] # subset dataframe to relevant information
        imf = pd.concat([imf, df]) # write the relevant information to imf

imf = imf.reset_index(drop=True)
os.makedirs('../temp', exist_ok=True) # Check for the directory
imf.to_csv('../temp/intermediate_mapping_file.csv', index = False)  # and create .csv file in the directory

In [120]:
## Dictionary creation for mappping -- necessary?? - yes, apparently


################################################################################


from pymongo import MongoClient
import pandas as pd

df = pd.read_csv('../temp/intermediate_mapping_file.csv')
# identifiers = ['Tumor_Sample_UUID','Matched_Norm_Sample_UUID','case_id']


client = MongoClient('localhost', 27017)
db = client.progenetixCopy ###############
bios_coll = db[ "biosamples" ]
ind_coll = db[ "individuals" ]

ind_ids = {}

tcga_s_id = df['Tumor_Sample_UUID']
tcga_mns_id = df['Matched_Norm_Sample_UUID']
tcga_c_id = df['case_id']

q = {'info.legacy_ids': tcga_s_id}


i_e_r = [
    {"id": "pgx:TCGA-"+tcga_c_id, "label": "TCGA case_id"},
]
for line in tcga_c_id:
    if line in ind_ids:
        ind_ids[line]["biosample_ids"].append(bios_coll.find_one({'info.legacy_ids': line})["id"]) #combined b an q from og code
    else:
        ind_ids.update({
            tcga_c_id: {
                "external_references": i_e_r,
                "biosample_ids": [b["id"]]
            }
        })

b_e_r = i_e_r.copy()
b_e_r.append({"id": "pgx:TCGA-"+tcga_s_id, "label": "TCGA sample_id"})

for e in b["external_references"]:
    if not "TCGA" in e["id"]: # <= important to keep other ers
        b_e_r.append(e)
    if re.match(r'^(?:pgx:)?TCGA\-\w+?$', e["id"]):
        project = re.sub('TCGA-', '', e["id"])
        project = re.sub('pgx:', '', project)
        b_e_r.append({"id": "pgx:TCGA-"+project, "label": "TCGA "+project+" project"})

if args.test:
    print(tcga_c_id, ind_ids[tcga_c_id]["biosample_ids"])
else:
    bar.next()
    bios_coll.update_one({"_id":b["_id"]}, {"$set": {"external_references": b_e_r}})


if not args.test:
    bar.finish()

############################################################################

TypeError: unhashable type: 'Series'

In [121]:
print(tcga_c_id)

0       becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
1       becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
2       becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
3       becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
4       becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
                        ...                 
1409    c632675e-e8af-43cd-8744-5be3754c1c14
1410    c632675e-e8af-43cd-8744-5be3754c1c14
1411    c632675e-e8af-43cd-8744-5be3754c1c14
1412    c632675e-e8af-43cd-8744-5be3754c1c14
1413    c632675e-e8af-43cd-8744-5be3754c1c14
Name: case_id, Length: 1414, dtype: object
